#Imports

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
import sys
import os
import pandas as pd
from skimage import io
from google.colab import drive
from PIL import Image

In [16]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
root_dir = '/content/gdrive/MyDrive/cats_and_dogs'

# Set Device

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [19]:
in_channel = 3
num_classes = 2
learning_rate = 0.001
batch_size = 64
num_epochs = 5

# Custom Dataset

In [20]:
class CatsAndDogsDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.img_size = 96
    self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)), 
            transforms.ToTensor()
        ])

  def __len__(self):
    return len(self.annotations)
  
  def __getitem__(self, index):
    img_path = os.path.join(self.root_dir, self.annotations.iloc[index,0])
    image =  Image.open(img_path)
    y_label = torch.tensor(int(self.annotations.iloc[index,1]))
    if self.transform:
      image = self.transform(image)
    return (image, y_label)

# Load Pretrain Model and Modification

In [21]:
model = torchvision.models.googlenet(weights="DEFAULT")

In [22]:
for param in model.parameters():
  param.requires_grad = False
model.fc = nn.Linear(in_features=1024, out_features=num_classes)
print(model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

# Load Data

In [23]:
img_size = 128

In [25]:
dataset = CatsAndDogsDataset(csv_file = root_dir+'/data.csv', root_dir = root_dir+'/train')
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 5000])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Initialize network

In [26]:
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

# Loss and optimizer

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Train Network

In [ ]:
for epoch in range(num_epochs):
  
  losses = []
  for batch_idx, (data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    # Forward
    scores = model(data)
    loss = criterion(scores, targets)
    losses.append(loss.item())

    # backward
    optimizer.zero_grad()
    loss.backward()

    # Gradient descent or adam step
    optimizer.step()
  mean_loss = sum(losses)/len(losses)
  print(f'Loss at epoch {epoch} was {mean_loss:.5f}')


#Check accuracy on training and test to see how good our model

In [ ]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct+=(predictions==y).sum()
      num_samples+=predictions.size(0)

    print(f'Got {num_correct/num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
  # model.train()



# Check Accuracy

In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)